In [1]:
%env CUDA_VISIBLE_DEVICES=3
%env MKL_NUM_THREADS=1

import sys
sys.path.insert(0, '/home/momoso/work_dir/pycommon')

env: CUDA_VISIBLE_DEVICES=3
env: MKL_NUM_THREADS=1


In [1]:
from datetime import datetime, timedelta
import random
import time
import numpy as np
from scipy import sparse
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
import clickhouse_driver as ck
import logging
import implicit
import matplotlib.pyplot as plt
import json
from tqdm.notebook import tqdm

import milvus
from milvus import Milvus, MetricType, IndexType
%matplotlib inline

In [4]:
np.__version__

'1.18.1'

In [3]:
pd.reset_option('all', 'use_inf_as_na')
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

In [9]:
def ck_client(host='localhost', port=49739):
    return ck.Client(host, port=port, database='recomm')

In [36]:
lg = logging.getLogger('als_server')
lg.setLevel(logging.DEBUG)


def show(df, col = 'rate'):
    num_user = df.anony_id.nunique()
    num_item = df.item_id.nunique()
    print('df shape', df.shape)
    print('users: ', num_user, '| items:', num_item)
    num_not_zero = (df[col] != 0).sum()
    zero_rate = (1 - num_not_zero / (num_user * num_item)) * 100
    print('sparsity: ', zero_rate, '%')
    print('rate count')
    print(df.rate.value_counts())


def get_rate_csr(df):
    item_cat = df.item_id.astype('category')
    user_cat = df.anony_id.astype('category')

    item_cate = list(item_cat.cat.categories)
    user_cate = list(user_cat.cat.categories)
    rates = list(df['rate'])
    columns = user_cat.cat.codes
    rows = item_cat.cat.codes

    rate_csr = sparse.csr_matrix((rates, (rows, columns)), shape=(len(item_cate), len(user_cate)))
    return rate_csr, item_cat, user_cat


def make_train(ratings, pct_test=0.3):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    parameters: 
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    returns:
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(42) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    item_inds = [index[0] for index in samples] # Get the user row indices
    user_inds = [index[1] for index in samples] # Get the item column indices
    training_set[item_inds, user_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered` 


def calc_mean_auc(training_set, test_set, altered_users, user_vecs, item_vecs):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    parameters:
    training_set - The training set resulting from make_train, where a certain percentage of the original
                    user/item interactions are reset to zero to hide them from the model 
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
                    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    test_set - The test set constucted earlier from make_train function
    returns:
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    store_aucs = [] # An empty list to store the AUC for each user that had an item removed from the training set
    pop_aucs = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 1)).reshape(-1) # Get sum of item iteractions to find most popular
    
    
    def cal_auc(user):
        training_rows = training_set[:, user].toarray().reshape(-1) # Get the training set row
        zero_inds = np.where(training_rows == 0) # Find where the interaction had not yet occurred
#         print('zero_inds ', len(zero_inds[0]), zero_inds)
        # Get the predicted values based on our user/item vectors
        user_vec = user_vecs[user,:]
        pred = user_vec.dot(item_vecs.T)
        pred = pred[zero_inds]
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[:,user][zero_inds,0].toarray().reshape(-1)
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        try:
            store_auc = metrics.roc_auc_score(actual, pred) # Calculate AUC for the given user and store
            popularity_auc = metrics.roc_auc_score(actual, pop) # Calculate AUC using most popular and score
            return store_auc, popularity_auc
        except Exception as e:
            print('actual', np.unique(actual))
            print(pred, pop)
        return 0,0
    
    for user in tqdm(altered_users):
        store_auc, pop_auc = cal_auc(user)
        if store_auc == 0 and pop_auc == 0:
            continue
        store_aucs.append(store_auc)
        pop_aucs.append(pop_auc)
    
    print('altered_users num', len(altered_users), 'auc num', len(store_aucs))
    return float('%.3f'%np.mean(store_aucs)), float('%.3f'%np.mean(pop_aucs))
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark


In [10]:
def load_data(start: str, end: str):
    try:
        client = ck_client()
        sql = ''
        sql = sql.replace('$start', "'{}'".format(start))
        sql = sql.replace('$end', "'{}'".format(end))
        rates = []
        rows = client.execute_iter(sql)
        for row in rows:
            rates.append(row)

        rates = pd.DataFrame(rates, columns=['item_id', 'anony_id', 'rate', 'time'])
        rates = rates.dropna(axis=0)
        print('rates shape', rates.shape)
        lg.debug('unique user: %s, unique item: %s', rates.anony_id.nunique(), rates.item_id.nunique())
    finally:
        client.disconnect()
    return rates


In [76]:
et = datetime.today() - timedelta(days=1)
st = et - timedelta(days=15)
st_str, et_str = st.strftime('%Y-%m-%d 00:00:00'), et.strftime('%Y-%m-%d 00:00:00')
st_str, et_str

('2020-07-27 00:00:00', '2020-08-11 00:00:00')

In [11]:
rates = load_data(st_str, et_str)

rates shape (558098, 4)


In [20]:
show(rates)

df shape (558098, 4)
users:  105251 | items: 130033
sparsity:  99.995922155573 %
rate count
1     476926
3      46318
2      19901
4       9898
10      3467
7       1588
Name: rate, dtype: int64


In [122]:
def subsampling(df, base=5):
    dfs = pd.DataFrame()
    for item_id, group in df.groupby(by='item_id'):
        rate = 1/(1+0.5*np.log(len(group)/base))
        num = int(len(group)*rate)
        idx = random.sample(group.index.tolist(), num)
        dfs = dfs.append(group.loc[idx])
    return dfs


def preprocess(df, user_th, item_th, user_rate_th=50):
    df = df.copy()
    show(df)
    ## filter user
    user_acts = rates.groupby('anony_id')['rate'].sum()
#     print(np.percentile(user_acts, [2, 50, 60, 80, 98, 100]))
    filter_users = set(list(user_acts[user_acts > user_rate_th].index))
    
    ## 每个人对多少商品有行为
    user_score = df.groupby(by='anony_id')['item_id'].count()
    invalid_users = set(list(user_score[user_score < user_th].index))
    filter_users = filter_users | invalid_users
#     df['anony_id'] = df['anony_id'].apply(lambda e: "UNKNOW_USER" if e in invalid_users else e)
#     df.replace(invalid_users, "UNKNOW_USER", inplace=True)  # too slow
    df = df[~df['anony_id'].isin(filter_users)]
    
    ## 每个商品被多少人行为过
    item_score = df.groupby(by='item_id')['anony_id'].count()
    invalid_items = set(list(item_score[item_score < item_th].index))
#     df.replace(invalid_items, "UNKNOW_ITEM", inplace=True)
    df = df[~df['item_id'].isin(invalid_items)]
#     df['item_id'] = df['item_id'].apply(lambda e: "UNKNOW_ITEM" if e in invalid_items else e)

    print('\nafter filter')
    show(df)
    return df


### get item info

In [8]:
random.seed(42)

In [123]:
new_rates = preprocess(rates, 5, 20) ## 每个人对多少商品有行为, 每个商品被多少人行为过

df shape (558098, 4)
users:  105251 | items: 130033
sparsity:  99.995922155573 %
rate count
1     476926
3      46318
2      19901
4       9898
10      3467
7       1588
Name: rate, dtype: int64

after filter
df shape (121195, 4)
users:  22396 | items: 1731
sparsity:  99.68737969941608 %
rate count
1     106841
3       9970
4       1944
2       1560
10       639
7        241
Name: rate, dtype: int64


In [119]:
# np.percentile(user_acts, [2, 50, 60, 80, 98, 100])

array([1.000e+00, 2.000e+00, 3.000e+00, 8.000e+00, 5.200e+01, 1.479e+03])

In [124]:
rate_csr, item_cate, user_cate = get_rate_csr(new_rates)
print('rate_csr shape', rate_csr.shape)

rate_csr shape (1731, 22396)


In [126]:
dim = 16

In [125]:
alpha = 40
train_d, val_d, altered_users = make_train(rate_csr)

In [127]:
t0 = time.time()
model = implicit.als.AlternatingLeastSquares(factors=dim, use_gpu=False, regularization=0.1, iterations=1000)
model.fit(train_d * alpha)
t1 = time.time()
print('fit consume: ', t1-t0, 's')

item_vecs = model.item_factors
user_vecs = model.user_factors


fit consume:  11.122735023498535 s


In [128]:
## 16
auc, pop_auc = calc_mean_auc(train_d, val_d, altered_users, user_vecs, item_vecs)
print('auc: %s, popular auc: %s'%(auc, pop_auc))


altered_users num 16842 auc num 16842
auc: 0.818, popular auc: 0.74


In [46]:
## 32
auc, pop_auc = calc_mean_auc(train_d, val_d, altered_users, user_vecs, item_vecs)
print('auc: %s, popular auc: %s'%(auc, pop_auc))


altered_users num 19472 auc num 19472
auc: 0.819, popular auc: 0.752


In [129]:
def recomm_by_item(item_id, k=20):
    item_categories = item_cate.cat.categories
    item_code = item_categories.get_loc(item_id)
    print('item id: %s, code:%s'%(item_id, item_code))
    print('item vec', item_vecs[item_code])

    if np.all(item_vecs[item_code] == 0):
        print('vectors of this item are all zero: %s', item_id)
        return None

    scores = cosine_similarity(item_vecs, [item_vecs[item_code]]).squeeze()
#     scores = scores / np.sum(scores)

    top_idx = np.argpartition(scores, -k)[-k:]
    top_k_items = sorted(zip(item_categories[top_idx], scores[top_idx]), key=lambda x: -x[1])
    return top_k_items


def recomm_by_user(anony_id, k=20):
    user_code = user_cate.cat.categories.get_loc(anony_id)
    user_vec = user_vecs[user_code]
    if np.all(user_vec == 0):
        print('vectors of this user are all zero: %s', anony_id)
        return None

    scores = user_vec.dot(item_vecs.T)
    items_viewed_codes = rate_csr[:,user_code].toarray().reshape(-1)
    item_categories = item_cate.cat.categories

    print('user viewed')
    items_viewed = item_categories[items_viewed_codes > 0]
    print(','.join([str(item) for item in items_viewed]))

    items_viewed_codes += 1
    items_viewed_codes[items_viewed_codes > 1] = 0

#     scaler = MinMaxScaler()
#     scores = scaler.fit_transform(np.expand_dims(scores, axis=1)).reshape(-1)
    scores = scores * items_viewed_codes

    top_idx = np.argpartition(scores, -k)[-k:]
    top_k_items = sorted(zip(item_categories[top_idx], scores[top_idx]), key=lambda x: -x[1])
    return top_k_items


## Evaluation

In [131]:
ids = item_cate.cat.categories.to_numpy().tolist()
vecs = normalize(item_vecs)

In [319]:
type(vecs.tolist())

list

In [208]:
np.random.choice(ids, 10)

array([35122981, 39484826, 14862917, 39712217, 28670060, 15875934,
       39319349, 39187207, 31736926, 39504699])

In [225]:
recalls = recomm_by_item(39712217, 100)
for i, recall in enumerate(recalls):
    print(i, recall)

item id: 39712217, code:1184
item vec [-0.00273029  0.14316387 -0.00772381 -0.00603587  0.12828703  0.03256284
  0.01543295  0.04750106 -0.10445275 -0.05162826 -0.03752997  0.13966726
 -0.0252912   0.04779462  0.10648493  0.01848328]
0 (39712217, 0.9999999)
1 (37438524, 0.7236471)
2 (36567287, 0.70498383)
3 (38289763, 0.70388824)
4 (39834259, 0.6923013)
5 (9616737, 0.68635833)
6 (39183498, 0.68084157)
7 (38844742, 0.6781229)
8 (13707649, 0.6705884)
9 (8108861, 0.6655506)
10 (38418232, 0.66426265)
11 (16265829, 0.6595068)
12 (22152775, 0.64300495)
13 (39922827, 0.6374818)
14 (37437569, 0.63503087)
15 (7795934, 0.6310046)
16 (38172144, 0.62399447)
17 (39022079, 0.62066346)
18 (36373301, 0.61896956)
19 (37227790, 0.61663723)
20 (39880770, 0.614757)
21 (16837766, 0.60944784)
22 (23290857, 0.6065514)
23 (39183326, 0.6050299)
24 (39365228, 0.6036536)
25 (39183297, 0.6013495)
26 (38120834, 0.59555197)
27 (13726338, 0.59361583)
28 (39587370, 0.59023964)
29 (39319044, 0.5834221)
30 (35887176, 0

## Explain

In [213]:
## 商品关联的人
users = new_rates.query('`item_id` == 39484826').anony_id.unique()
print(len(users))

57


In [214]:
## 这些人的行为
acts = new_rates[new_rates.anony_id.isin(set(users))]

In [215]:
## 每个人每种评分对多少个品
act_users = acts.groupby(['anony_id', 'rate'])['item_id'].count().sort_values(ascending=False)
act_users

anony_id                          rate
79fe315d27bd434985fd0e9c281c162a  1       29
d91f6d22ff684b098047be6a18fab4e9  1       22
0be1e4549fed432dbdfdf6d1a20af5e2  1       21
60075245a2044f96ace4a34c46179b0f  1       17
57c09edac4964b6ab9925ec79ba63300  1       15
                                          ..
974a2161cb454a79807232ad3ad8e446  1        1
612bb279cad94e1c8d544525aed1b3cb  3        1
5546d001588c44249491bc7e6dbf7ff8  10       1
d70d890062e8429c84b35793e20cb7d6  3        1
                                  4        1
Name: item_id, Length: 91, dtype: int64

In [216]:
## 行为中商品评分降序
act_items = acts.groupby('item_id')['rate'].sum().sort_values(ascending=False)
act_items

item_id
39484826    81
39484769    27
39484623    20
39834214    15
26262505    14
            ..
39444310     1
39437768     1
39427530     1
39427430     1
7552698      1
Name: rate, Length: 236, dtype: int64

## 共现

In [302]:
recomm_items = [40032909,6948864]
v = new_rates[new_rates.item_id.isin(recomm_items)]
pivots = v.pivot(index='item_id', columns='anony_id', values='rate').fillna(0)

In [303]:
pivots

anony_id,00c80bb78bd34dc78384ab2e1d431f60,028a2b8b2be448a3840a5827985475a8,05caab2c3d7a4b6b9e2ec66a15cc5ac5,08313b01a55b4b9aa5b09118e67d6bc5,09432c094d6e451bb0995f84cbf1fa85,11c2503fb3a24c8e97272494f9f26469,129281bf2f834cec9ffb0ee0a3d4c747,188d2820000842b88a5b0d9f077f0e82,18e55624af2b4759b62df20a3eccd196,1903c0a10acf4908a7776c1bc6b5d2e1,19e121a9e40b4d6d89052cdb8c9f6718,1ded8155caa14f18bff0b44264c5d950,1f09e4b1cd284f839d0fb3f58adc3228,20a43ff8b10e49f48b680cad757489ff,21f9107ed50e4484a618298e33e3fd34,22262f7684c640a8b9a718b82abed9e4,26b95dabc7c64bfbb24ffe1a07ed7d19,2cc259782d2443189600939297892d62,31c447cce96545419ba8163351f5a786,3313d88491c54253a89668fc61435a72,346b6da53db94a64a97143e637e73e21,44cd9c5969344824aca1fa52c5daab08,46c1ee40941a473d94ff42970739d02f,494733ab9da841db8ac3365ec4e32475,4c45c3f1c47145b190cdfaf4818d5995,4c52467573a34364b694a94ddd0a9b37,511bb80152b240ac982768ccf306c352,531ad748fd2a4bea888838f8e759c1f1,537163e6f7f641e8a17a4ee9878f5074,562f5c057eb0493eb4ab97734557a5cb,5d24c5e503694ed492b9dfefabf66243,60937800132e42b8bbc42a7f69961f45,6bce4a9fb4fb4826bcf3090a8873d42f,6be59faac20245c6958aae4a8c7149c6,747effccf5394519874596d2fdf6390c,76cd3882389c4c0baee04f7704d82003,77f3d98cffc34c9da4bcfff3a72e244e,79599915956e4c589ae57c10c10a85ee,7e8185ef4ffb4778815034ff71a12614,85cf64553999400796daabe4baae2f5d,8907c9b0a50540738c7c376468281a08,8c1d15c2a6224c1b85af204598b1609d,927dc193384547c99f6bd395d612ec9e,94a67ce8aedc453fbfe9e14e1d9478aa,982cb723775c4ca79a089ab3aa966af0,9d80581f72ce4b729db15c4ebf72a1c6,9f4c43a12de54d44bc5cf86777c35574,a1d9a2e1b13c4e508159ea8adc91be6d,a1efb76e0de24538bcb28bf59e015b37,a5e30e239c0645f0892cf56b2dd50146,a75fa88de07c4b08bde9c3b3137b82ae,adf62eb733a44706a177dad2878701fb,b4ed195491a146468de107f4e60537b1,b54494f53e6f4db9919492624bcf7c92,b671f1707777487d96ec7d7e6cf31711,b74b5aaaec7f42dab13367984d90565e,bcc17ede8c754b38a871a7ac8aec73ff,bcd65c3186ff4e8e803e5593d121b0c6,c502a905304e40f3a259534f23d15c29,c57ab092260a4be3bc367539167e7ae9,c82cc072d3d742c09881beba8d86fbc4,c9ca40e5b6a4415aa0cdb23f334f529b,cd2cfcbb62654a91a7b547f94124fcd4,d60903d815554dd39301649c28587cc6,d8c031797b8d466baf619dce826222cb,dd5ecfdf1b634757bbbd422786aeed83,e63b6255e67e418f896194c6c25aecbb,fafd5992416c45f487e57759624d0158,fdb20a23a4eb4a1aa5db6fb4773f3b37,fe05a6f016ff4b51b14c31eefcbd45b2
item_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6948864,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
40032909,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0,3.0,1.0,3.0,1.0,0.0,0.0


In [ ]:
pivots.loc[:, np.all(pivots > 0, axis=0)]

In [299]:
users = new_rates[new_rates.anony_id.isin(v.anony_id.unique())]

In [301]:
p = users.pivot(index='item_id', columns='anony_id', values='rate').fillna(0)

anony_id,00c80bb78bd34dc78384ab2e1d431f60,028a2b8b2be448a3840a5827985475a8,05caab2c3d7a4b6b9e2ec66a15cc5ac5,08313b01a55b4b9aa5b09118e67d6bc5,09432c094d6e451bb0995f84cbf1fa85,11c2503fb3a24c8e97272494f9f26469,129281bf2f834cec9ffb0ee0a3d4c747,188d2820000842b88a5b0d9f077f0e82,18e55624af2b4759b62df20a3eccd196,1903c0a10acf4908a7776c1bc6b5d2e1,19e121a9e40b4d6d89052cdb8c9f6718,1ded8155caa14f18bff0b44264c5d950,1f09e4b1cd284f839d0fb3f58adc3228,20a43ff8b10e49f48b680cad757489ff,21f9107ed50e4484a618298e33e3fd34,22262f7684c640a8b9a718b82abed9e4,26b95dabc7c64bfbb24ffe1a07ed7d19,2cc259782d2443189600939297892d62,31c447cce96545419ba8163351f5a786,3313d88491c54253a89668fc61435a72,346b6da53db94a64a97143e637e73e21,44cd9c5969344824aca1fa52c5daab08,46c1ee40941a473d94ff42970739d02f,494733ab9da841db8ac3365ec4e32475,4c45c3f1c47145b190cdfaf4818d5995,4c52467573a34364b694a94ddd0a9b37,511bb80152b240ac982768ccf306c352,531ad748fd2a4bea888838f8e759c1f1,537163e6f7f641e8a17a4ee9878f5074,562f5c057eb0493eb4ab97734557a5cb,5d24c5e503694ed492b9dfefabf66243,60937800132e42b8bbc42a7f69961f45,6bce4a9fb4fb4826bcf3090a8873d42f,6be59faac20245c6958aae4a8c7149c6,747effccf5394519874596d2fdf6390c,76cd3882389c4c0baee04f7704d82003,77f3d98cffc34c9da4bcfff3a72e244e,79599915956e4c589ae57c10c10a85ee,7e8185ef4ffb4778815034ff71a12614,85cf64553999400796daabe4baae2f5d,8907c9b0a50540738c7c376468281a08,8c1d15c2a6224c1b85af204598b1609d,927dc193384547c99f6bd395d612ec9e,94a67ce8aedc453fbfe9e14e1d9478aa,982cb723775c4ca79a089ab3aa966af0,9d80581f72ce4b729db15c4ebf72a1c6,9f4c43a12de54d44bc5cf86777c35574,a1d9a2e1b13c4e508159ea8adc91be6d,a1efb76e0de24538bcb28bf59e015b37,a5e30e239c0645f0892cf56b2dd50146,a75fa88de07c4b08bde9c3b3137b82ae,adf62eb733a44706a177dad2878701fb,b4ed195491a146468de107f4e60537b1,b54494f53e6f4db9919492624bcf7c92,b671f1707777487d96ec7d7e6cf31711,b74b5aaaec7f42dab13367984d90565e,bcc17ede8c754b38a871a7ac8aec73ff,bcd65c3186ff4e8e803e5593d121b0c6,c502a905304e40f3a259534f23d15c29,c57ab092260a4be3bc367539167e7ae9,c82cc072d3d742c09881beba8d86fbc4,c9ca40e5b6a4415aa0cdb23f334f529b,cd2cfcbb62654a91a7b547f94124fcd4,d60903d815554dd39301649c28587cc6,d8c031797b8d466baf619dce826222cb,dd5ecfdf1b634757bbbd422786aeed83,e63b6255e67e418f896194c6c25aecbb,fafd5992416c45f487e57759624d0158,fdb20a23a4eb4a1aa5db6fb4773f3b37,fe05a6f016ff4b51b14c31eefcbd45b2
item_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6948864,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
7552698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7757978,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7795934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8931751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [275]:
v.groupby('item_id')['rate'].sum()

item_id
6948864     27
40032909    61
Name: rate, dtype: int64

In [282]:
vs = new_rates.groupby('item_id')['rate'].sum()
vs

item_id
6948864     27
7024575     51
7031063     28
7538009     29
7549191     59
            ..
40008562    46
40010307    46
40010308    80
40010857    50
40032909    61
Name: rate, Length: 1731, dtype: int64

In [295]:
np.percentile(vs.values, range(0,110,10))

array([  20.,   27.,   31.,   36.,   42.,   50.,   61.,   78.,  115.,
        182., 1946.])

## 人的行为

In [196]:
new_rates.query("`anony_id` == 'bf044177ae4446209df940209e3f7677'")

,item_id,anony_id,rate,time
243184,21729564,bf044177ae4446209df940209e3f7677,3,2020-07-27 02:00:58
314598,39372630,bf044177ae4446209df940209e3f7677,1,2020-07-27 02:13:34
394695,39364397,bf044177ae4446209df940209e3f7677,1,2020-07-27 02:12:27


In [153]:
import gc
gc.collect()

45516